In [1]:
import torch
import matplotlib.pyplot as plt
import pickle
import pandas as pd

from functions import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_util import RAdam,trainable_parameter

In [2]:
sz = 256
batch_size = 8
epochs = 10
clip = 1.0
lr = 4e-4
encoder_str = 'efficientnet-b2'
decoder_channels=(256, 128, 64, 32, 16)
decoder_repeats=(2,2,2,2,2)
best_threshold = 0.5
min_size = 3500
includeX0 = False
opt_level="O1"

In [3]:
images_dir = "../Data/pickles_"+str(sz)+"/images/"
masks_dir = r"../Data/pickles_"+str(sz)+"/masks/"

In [4]:
imageId = pd.read_csv('../Data/stage_2_train.csv')['ImageId'].values.tolist()
np.random.shuffle(imageId)

In [5]:
# setup train/val
imageId_val = imageId[:2400]
imageId_train = imageId[2400:]

In [6]:
if sz in preprocessing_dict:
    dataset_train = dataset(imageId_train,images_dir,masks_dir,transform,preprocessing_dict[sz])
else:
    dataset_train = dataset(imageId_train,images_dir,masks_dir,transform)
    dataset_train._cal_preprocessing()
dataset_val = dataset(imageId_val,images_dir,masks_dir,preprocessing=dataset_train.preprocessing)

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=4)

In [7]:
# imgs,masks = next(iter(train_loader))

In [8]:
# freeze encoder
encoder = EfficientNet_encoder.from_pretrained(encoder_str,includeX0=includeX0)
set_requires_grad(encoder,False)
decoder = EfficientNet_decoder(sz,encoder_channels[encoder_str],decoder_channels=decoder_channels,decoder_repeats=decoder_repeats)
model = Unet(encoder,decoder).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-05)
model, opt = amp.initialize(model, opt, opt_level=opt_level)

Loaded pretrained weights for efficientnet-b2
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [9]:
model,bestWeight,bestOpt,bestAmp = train(opt,model,epochs,train_loader,\
                                 valid_loader,paras,clip,scheduler=scheduler)

epoch:0, train_loss: +1.304, val_loss: +1.208

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
epoch:1, train_loss: +0.816, val_loss: +0.680

epoch:2, train_loss: +0.741, val_loss: +0.658

epoch:3, train_loss: +0.716, val_loss: +0.678

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
epoch:4, train_loss: +0.699, val_loss: +0.640

epoch:5, train_loss: +0.690, val_loss: +0.635

epoch:6, train_loss: +0.686, val_loss: +0.641

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
epoch:7, train_loss: +0.670, val_loss: +0.659

epoch:8, train_loss: +0.662, val_loss: +0.620

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
epoch:9, train_loss: +0.654, val_loss: +0.594

Training completed in 3952.7846598625183s


In [10]:
checkpoint = {
    'model': bestWeight,
    'opt': bestOpt,
    'amp': bestAmp
}
torch.save(checkpoint, '../Model/'+str(sz)+'_'+encoder_str+'_'+str(decoder_channels)+'_'+str(decoder_repeats)+'_pretrain.pt')

In [ ]:
# checkpoint = torch.load('amp_checkpoint.pt')
# model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
# model.load_state_dict(checkpoint['model'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# amp.load_state_dict(checkpoint['amp'])

In [ ]:
# unfreeze encoder with opt reset
set_requires_grad(encoder,True)
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr*0.25,weight_decay=1e-2)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-05*0.1)
model, opt = amp.initialize(model, opt, opt_level=opt_level)

model,bestWeight,bestOpt,bestAmp = train(opt,model,epochs,train_loader,valid_loader,paras,clip,scheduler=scheduler)

In [ ]:
# unfreeze encoder
set_requires_grad(encoder,True)
opt.add_param_group({'params': encoder.parameters()})
for param_group in opt.param_groups:
    param_group['lr'] = lr * 0.25
model,bestWeight,bestOpt,bestAmp = train(opt,model,epochs,train_loader,valid_loader,paras,clip,scheduler=scheduler)

Score test data

In [11]:
df = pd.read_csv('../Data/stage_2_sample_submission.csv')
testID = df['ImageId'].values.tolist()
dataset_test = dataset(testID,images_dir,preprocessing=dataset_train.preprocessing)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=4)

In [12]:
yhat = predict(model,test_loader)

In [13]:
submit(df,yhat,best_threshold, min_size,\
       '../Submission/'+str(sz)+'_'+encoder_str+'_'+str(decoder_channels)+'_'+str(decoder_repeats)+'.csv')